In [119]:
import numpy as np
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
import json
from sklearn.metrics import mean_squared_error
from random import sample
import time


In [2]:
#從xlsx匯入資料
rec_data= pd.read_excel(r'./melt_data.xlsx',encoding='utf-16',index_col=0)

In [3]:
#sample train test data

train_idx=sample(rec_data.index.to_list(),int(len(rec_data)*0.7))
train_data=rec_data.loc[train_idx]
test_tmp=rec_data.index.isin(train_idx)
test_data=rec_data[~test_tmp]

In [4]:
train_data.head()

,user,variable,value
311494,1561,美國運通-簽帳金卡,0
407164,1867,新光銀行-Lamigo信用卡,0
445558,526,臺灣企銀-鈦金商旅卡,0
378471,2313,渣打銀行-TheShoppingCard分期卡,0
223744,1228,玉山銀行-麗寶悠遊聯名卡,0


In [11]:
test_data.head()

,user,variable,value
0,0,三信銀行-VISA白金卡,0
2,2,三信銀行-VISA白金卡,0
4,4,三信銀行-VISA白金卡,0
5,5,三信銀行-VISA白金卡,0
6,6,三信銀行-VISA白金卡,0


In [8]:
#melt data to df and fillna with 0
train_df=train_data.pivot_table(values='value',index='user',columns='variable').fillna(0)
train_df.head()

variable,三信銀行-VISA白金卡,上海商銀-TeresaCard,上海商銀-小小兵Bello回饋卡,上海商銀-簡單卡,中國信託-ANA聯名卡,中國信託-LINE Pay卡,中國信託-MUJI無印良品聯名卡,中國信託-Yahoo聯名卡,中國信託-中信兄弟聯名卡,中國信託-中信商旅鈦金卡,...,華南銀行-i網購生活卡,華南銀行-享利樂活Combo卡,華南銀行-大甲媽祖認同卡,華南銀行-旅鑽商務御璽卡,華南銀行-美饌紅利卡,華南銀行-超級現金回饋卡,遠東商銀-Bankee信用卡,遠東商銀-快樂信用卡,陽信銀行-一卡通聯名鈦金卡,陽信銀行-曜晶卡
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
indices = pd.Series(train_df.columns)   #所有卡series
#card_index_list=indices[train_df.loc[1]==1]

print(train_df.columns[train_df.loc[0]==1].to_list())
print(train_df.columns[train_df.loc[1]==1].to_list())

['中國信託-LINE Pay卡', '台新銀行-FlyGo卡']
['中國信託-LINE Pay卡', '台新銀行-@GoGo iCash御璽卡', '花旗銀行-新世代花旗饗樂生活悠遊卡']


In [75]:
#test data total card
len(test_data[test_data['value']==1])/len(test_data)

0.022097786725948296

# Item_Based_Evaluate

In [124]:
#item_base_rec
def CF_Item_Based_Evaluate(train,TopN):
    start_time = time.time()

    #imput data type:df
    #轉置df
    traindata=train
    traindata_T=traindata.T
    #計算各卡間的相似度
    card_sim=cosine_similarity(traindata_T,traindata_T)
    
    #找出已持有卡index
    item_rec_list={}
    for user in traindata.index:
           #所有卡series
        my_card=traindata.columns[traindata.loc[user]==1].to_list()#找持有卡
        indices = pd.Series(traindata_T.index)
        card_index_list=[indices[indices == name].index[0] for name in my_card]  #找出各卡index
        weighted_card_rec=np.zeros(len(traindata_T.index))   #創全0且為卡片數量長度的array
        for  i in card_index_list:
            weighted_card_rec +=card_sim[i]           #將持有卡於其他卡的相似度相加成推薦權重
        #排列相似度最高前TopN名卡片
        weighted_score=pd.Series(weighted_card_rec).sort_values(ascending = False)
        TopN_indexes=weighted_score.iloc[len(my_card):len(my_card)+TopN].index
        rec_card=[]
        for i in TopN_indexes:
            rec_card.append(indices[i])
        item_rec_list[user]=rec_card
    #calculate TopN in test_data
    #test_data user
    total_precision=0
    total_hit =0
    result={}

    for user in list(set(test_data['user'])):
        user_df=test_data[test_data['user']==user]
        user_df_hold=user_df[user_df['value']==1]['variable'].values
        hit=sum([1 for item in item_rec_list[user] if item in user_df_hold])
        precision=hit/TopN
        
        total_precision +=precision
        total_hit +=hit
    result['Precision']=total_precision/len(set(test_data['user']))
    
    result['Hit']=total_hit/len(set(test_data['user']))
    exec_time= "--- %s seconds ---" % (time.time() - start_time)


    return result,exec_time


In [125]:
CF_Item_Based_Evaluate(train_df,5)

({'Precision': 0.10872027180067793, 'Hit': 0.5436013590033975},
 '--- 12.733741283416748 seconds ---')

In [109]:
len(test_data[test_data['value']==1])

3161

#  User_Based

In [127]:
def CF_User_Based_Evaluate(train,NeighborN,TopN):
    #imput data type:my_card=['上海商銀-小小兵Bello回饋卡','中國信託-ANA聯名卡']
    start_time = time.time()
    traindata=train
    

    #user與其他user計算cosine相似度
    user_sim_array=cosine_similarity(train_df,train_df)
    user_rec_list={}
    for user in traindata.index:
        sim_value_sort=pd.Series(user_sim_array[user]).sort_values(ascending = False)
    # df_m2m = pd.DataFrame(cosine_similarity(df))
        topN_sim_user={}
        
        for idx,v in sim_value_sort.iteritems():
            if len(topN_sim_user)<NeighborN:     #topN_sim_user
                topN_sim_user[idx]=v  

        weighted_card_rec=np.zeros(len(traindata.columns))
        #相似權重相加
        for i in topN_sim_user:
            #print(i)
            weighted_card_rec+= traindata.loc[i]*topN_sim_user[i]
              
        #未持有的卡推薦  
        recommend_card=[]
        weighted_card_rec_sort=weighted_card_rec.sort_values(ascending=False)
        for card,value in weighted_card_rec_sort.iteritems():
                if value >0:
                    if train_df.loc[user,card]==0 and len(recommend_card) <TopN:
                                      recommend_card.append(card)
        user_rec_list[user]=recommend_card
    total_precision=0
    total_hit =0
    result={}

    for user in list(set(test_data['user'])):
        user_df=test_data[test_data['user']==user]
        user_df_hold=user_df[user_df['value']==1]['variable'].values
        hit=sum([1 for item in user_rec_list[user] if item in user_df_hold])
        precision=hit/TopN
        
        total_precision +=precision
        total_hit +=hit
    result['Precision']=total_precision/len(set(test_data['user']))
    result['Hit']=total_hit/len(set(test_data['user']))
    exec_time="--- %s seconds ---" % (time.time() - start_time)

    return result,exec_time
                    

In [128]:
CF_User_Based_Evaluate(train_df,100,5)

#train_df.loc[user,card]==0
#user_df=train_df[train_df['user']==1]
#user_df['中國信託-ANA聯名卡'] == 1

({'Precision': 0.10856927142317693, 'Hit': 0.5428463571158928},
 '--- 118.11028790473938 seconds ---')

In [130]:
CF_User_Based_Evaluate(train_df,5,5)

({'Precision': 0.033069082672706814, 'Hit': 0.16534541336353342},
 '--- 14.566324710845947 seconds ---')

In [ ]:


reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'title', 'rating']], reader)
trainSet, testSet = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(random_state=0)
algo.fit(trainSet)
predictions = algo.test(testSet)

def MAE(predictions):
        return accuracy.mae(predictions, verbose=False)
def RMSE(predictions):
        return accuracy.rmse(predictions, verbose=False)
    
print("RMSE: ", RMSE(predictions))
print("MAE: ", MAE(predictions))

In [3]:
SVD

NameError: name 'SVD' is not defined